In [1]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from scipy.spatial.distance import pdist,squareform
from sklearn.decomposition import PCA
import torch
import transformers as ppb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

# Dimension reduction and clustering libraries
import umap
import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\basuc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def create_attention_mask(input_id):
    attention_masks = []
    for sent in input_ids:
        att_mask = [int(token_id > 0) for token_id in sent]  # create a list of 0 and 1.
        attention_masks.append(att_mask)  # basically attention_masks is a list of list
    return attention_masks

In [3]:
# define mean pooling function
def mean_pool(token_embeds, attention_mask):
    # reshape attention_mask to cover 768-dimension embeddings
    in_mask = attention_mask.unsqueeze(-1).expand(
        token_embeds.size()
    ).float()
    # perform mean-pooling but exclude padding tokens (specified by in_mask)
    pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(
        in_mask.sum(1), min=1e-9
    )
    return pool

In [4]:
def arr_largest(arr,n):
    max = arr[0]
    for i in range(1, n):
        if arr[i] > max:
            max = arr[i]
    return max

In [5]:
def arr_smallest(arr,n):
    min = arr[0]
    for i in range(1, n):
        if arr[i] < min:
            min = arr[i]
    return min

In [6]:
bert_model_class, bert_tokenizer_class, bert_pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
##PATH = './bert_uncased_save'
bert_tokenizer = bert_tokenizer_class.from_pretrained(bert_pretrained_weights)
bert_model = bert_model_class.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
t5_path = './model_files_t52'
t5_model = T5ForConditionalGeneration.from_pretrained(t5_path)
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

In [8]:
import datasets
train_data = datasets.load_dataset("ccdv/cnn_dailymail", "3.0.0", split="test[0:100]")
article_list = []
highlight_list = []
id_list = []
bert_output_list =[]
output_list=[]
for i in range(len(train_data)):
    article_str = train_data['article'][i]
    highlight_str = train_data['highlights'][i]
    id_str = train_data['id'][i]
    article_list.append(article_str)
    highlight_list.append(highlight_str)
    id_list.append(id_str)
train_df = pd.DataFrame(columns=['text','headline'])
train_df['text']=article_list
train_df['headline']=highlight_list

Reusing dataset cnn_dailymail (C:\Users\basuc\.cache\huggingface\datasets\ccdv___cnn_dailymail\3.0.0\3.0.0\0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f)


In [9]:
final_highlight_list = []
number_extract = 11
for index, row in train_df.iterrows():
    paragraph = row['text']
    paragraph_split = sent_tokenize(paragraph)
    
    if len(paragraph_split) < number_extract:
        continue
    else:
        input_tokens = []
        for i in paragraph_split:
            input_tokens.append(bert_tokenizer.encode(i,
                                add_special_tokens=True))

        temp = []
        for i in input_tokens:
            temp.append(len(i))
        max_sentence_length = np.max(temp)
        #print("max sentence length "+str(max_sentence_length))

        input_ids = pad_sequences(
            input_tokens,
            maxlen=max_sentence_length + 10,
            dtype='long',
            value=0,
            truncating='post',
            padding='post',
            )

        input_masks = create_attention_mask(input_ids)

        input_ids = torch.tensor(input_ids)
        attention_mask = torch.tensor(input_masks)

        with torch.no_grad():
            last_hidden_states = bert_model(input_ids, attention_mask=attention_mask)

        #token_embed = last_hidden_states[0]
        #x_final = mean_pool(token_embed, attention_mask)
        #sentence_features = x_final.detach().numpy()
        sentence_features = last_hidden_states[0][:,0,:].detach().numpy()

        #umap_embedding = umap.UMAP(n_neighbors=6, min_dist=0.0, n_components=2,
                                   #random_state=42).fit_transform(sentence_features)
        kmeans = KMeans(n_clusters=number_extract,random_state=0).fit(sentence_features)
        cluster_center = kmeans.cluster_centers_
        nbrs = NearestNeighbors(n_neighbors= 1,algorithm='brute').fit(sentence_features)
        distances, indices = nbrs.kneighbors(cluster_center.reshape(number_extract,-1))

    #     hdbscan_labels = hdbscan.HDBSCAN(min_samples=2,
    #                                      min_cluster_size=2).fit_predict(umap_embedding)

    #     n = len(hdbscan_labels)
    #     max_label = arr_largest(hdbscan_labels, n)
    #     min_label = arr_smallest(hdbscan_labels, n)
    #     number_extract = (max_label - min_label)+1


    #     centroid_arr = []
    #     for i in range(min_label, max_label + 1):
    #         cluster_centroid = np.mean(sentence_features[hdbscan_labels == i, :
    #                                    ], axis=0)

    #         centroid_arr.append(cluster_centroid)


    #     centroid_arr = np.stack(centroid_arr, axis=0)


    #     nbrs = NearestNeighbors(n_neighbors=1, algorithm='brute'
    #                             ).fit(sentence_features)
    #     (distances, indices) = \
    #         nbrs.kneighbors(centroid_arr.reshape(number_extract, -1))

        indices = np.sort(indices.reshape(1, -1))


        topic_answer = []
        indices_list =[]
        for i in indices[0]:
            if i not in indices_list:
                topic_answer.append(paragraph_split[i])
                indices_list.append(i)

        topic_answer_string = ''
        for topic in topic_answer:
            topic_answer_string = topic_answer_string + ' ' + topic

        text = topic_answer_string
        preprocess_text = text.strip().replace('\n', '')
        t5_prepared_Text = 'summarize: ' + preprocess_text

        tokenized_text = t5_tokenizer.encode(t5_prepared_Text,
                return_tensors='pt').to(device)

        summary_ids = t5_model.generate(
                                        tokenized_text,
                                        max_length=256,
                                        num_beams=5,
                                        repetition_penalty=2.5,
                                        length_penalty=1.0,
                                        early_stopping=False,
                                        )

        output = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        output_list.append(output)
        bert_output_list.append(topic_answer_string)
        final_highlight_list.append(row['headline'])


In [10]:
output_df = pd.DataFrame(columns=['bert_headline','generated_headline','headline'])
output_df['bert_headline'] = bert_output_list
output_df['generated_headline'] = output_list
output_df['headline'] = final_highlight_list
output_df.to_csv('test_metrics_kmeans10_or_bert.csv')

In [11]:
from datasets import load_metric
metric = load_metric("rouge")
preds = output_list
labels = final_highlight_list
metric.compute(predictions=preds, references=labels)

{'rouge1': AggregateScore(low=Score(precision=0.2918220240747813, recall=0.08746464272045197, fmeasure=0.12932378984987963), mid=Score(precision=0.33620002733513743, recall=0.10385774979452084, fmeasure=0.15096112819199747), high=Score(precision=0.3827850214362163, recall=0.12099475069476162, fmeasure=0.17323661156170178)),
 'rouge2': AggregateScore(low=Score(precision=0.08745205179488266, recall=0.02503703320997465, fmeasure=0.037615188361750804), mid=Score(precision=0.12393399538753157, recall=0.03605836876284827, fmeasure=0.053190198900247246), high=Score(precision=0.16501576408730406, recall=0.04889723946646475, fmeasure=0.07090767783526027)),
 'rougeL': AggregateScore(low=Score(precision=0.24090247335640244, recall=0.07133050223403557, fmeasure=0.10608550368986815), mid=Score(precision=0.2773563390245375, recall=0.08420985041568005, fmeasure=0.1229771380669226), high=Score(precision=0.32217212716902505, recall=0.0988908430799275, fmeasure=0.14386038848343383)),
 'rougeLsum': Aggre

In [12]:
preds = bert_output_list
labels = final_highlight_list
metric.compute(predictions=preds, references=labels)

{'rouge1': AggregateScore(low=Score(precision=0.08926043326897191, recall=0.5599170453851255, fmeasure=0.15252556747016205), mid=Score(precision=0.09597015755601529, recall=0.5916071632673928, fmeasure=0.16254482432993478), high=Score(precision=0.10315975856187229, recall=0.6223223031664806, fmeasure=0.1738614305588333)),
 'rouge2': AggregateScore(low=Score(precision=0.028267862767387807, recall=0.18736118163554766, fmeasure=0.04833081650971485), mid=Score(precision=0.033030475688133276, recall=0.21746112141769341, fmeasure=0.05640677840813618), high=Score(precision=0.03799353192979721, recall=0.24995042326188105, fmeasure=0.06481234888250491)),
 'rougeL': AggregateScore(low=Score(precision=0.05834061607451642, recall=0.3717879220993162, fmeasure=0.09995807740399709), mid=Score(precision=0.06335849636524715, recall=0.4010873649654602, fmeasure=0.1075468463183145), high=Score(precision=0.06782842226964816, recall=0.43158587696355805, fmeasure=0.11457506857072851)),
 'rougeLsum': Aggrega